In [1]:
import re
import os

class LogRecord:
    def __init__(self, timestamp: str, level: str, source: str, message: str,):
        self.timestamp = timestamp
        self.level = level
        self.source = source
        self.message = message
    def __str__(self):
        return f"{self.timestamp} {self.level} [{self.source}] {self.message}"

class LogParser:
    def_pattern = r'^(?P<timestamp>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}) (?P<level>\w+) \[(?P<source>\w+)\] (?P<message>.+)$'
    def __init__(self, pattern: str):
         if pattern:
              self.pattern = re.compile(pattern)
         else:
              self.pattern = re.compile(self.def_pattern)
    def parse_line(self, line: str) -> LogRecord|None:
         match = self.pattern.match(line.strip())
         if match:
              return LogRecord(match.group('timestamp'), match.group('level'), match.group('source'), match.group('message'))
         else:
             return None


class LogAnalyzer:
    def __init__(self, filepath: str, parser: LogParser):
            self.filepath = filepath
            self.parser = parser
            self.record_list = []
    def load(self):
         if not os.path.exists(self.filepath):
              print("Файл не найден")
              return
         with open(self.filepath, 'r', encoding='utf-8') as f:
              for line in f:
                   if line.strip():
                        record = self.parser.parse_line(line)
                        if record:
                             self.record_list.append(record)                
    def filter_by_level(self, level: str) -> list[LogRecord]:
         return [record for record in self.record_list if record.level == level]
    def filter_by_source(self, source: str) -> list[LogRecord]:
         return [record for record in self.record_list if record.source == source]
    def search_in_message(self, pattern: str) -> list[LogRecord]:
         regex = re.compile(pattern)
         return [record for record in self.record_list if regex.search(record.message)]

parser = LogParser(pattern = "")
logs_app = LogAnalyzer("logs.txt", parser)
logs_app.load()
print("Записи с уровнем ERROR")
info = logs_app.filter_by_level("INFO")
for i in info:
     print(i)
print("Записи с источником db")
db = logs_app.filter_by_source("db")
for i in db:
     print(i)
print("Записи, которые содержат ip-адрес")
ip = logs_app.search_in_message(r'\d{3}.\d{3}.\d{1}.\d{2}')
for i in ip:
     print(i)

Записи с уровнем ERROR
2025-12-01 12:45:32,123 INFO [auth] User "alex" logged in from 192.168.0.10
2025-12-01 12:45:32,123 INFO [db] User "jane" logged in from 192.168.0.10
Записи с источником db
2025-12-01 12:46:01,987 ERROR [db] Connection timeout for query "SELECT * FROM users"
2025-12-01 12:45:32,123 INFO [db] User "jane" logged in from 192.168.0.10
Записи, которые содержат ip-адрес
2025-12-01 12:45:32,123 INFO [auth] User "alex" logged in from 192.168.0.10
2025-12-01 12:45:32,123 INFO [db] User "jane" logged in from 192.168.0.10
